In [1]:
# !pip install --quiet langgraph langgraph-supervisor langchain-openai

In [2]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

In [ ]:
model = ChatOpenAI(temperature=0, api_key="", model="gpt-4o-mini")

In [4]:
import chromadb

client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_collection("pharma-kb")


In [5]:
def search_pharma_kb(query, top_n=10):
    results = collection.query(
        query_texts=[query],
        n_results=top_n
    )

    return {"documents": results["documents"]}

In [6]:
query = "What are some alternatives to Ibuprofen for pain relief"
print(search_pharma_kb(query, 10))

{'documents': [['of NSAIDs with aspirin (seePRECAUTIONS;Drug Interactions).Clinical StudiesIn controlled, double-blind, clinical trials, mefenamic acid was evaluated for the treatment of primary spasmodic dysmenorrhea. The parameters used in determining efficacy included pain assessment by both patient and investigator; the need for concurrent analgesic medication; and evaluation of change in frequency and severity of symptoms characteristic of spasmodic dysmenorrhea. Patients received either mefenamic acid,', 'DysmenorrheaIn acute analgesic models of post-oral surgery pain, post-orthopedic surgical pain, and primary dysmenorrhea, celecoxib relieved pain that was rated by patients as moderate to severe. Single doses [seeDosage and Administration (2.6)] of celecoxib provided pain relief within 60 minutes.14.6  Cardiovascular Outcomes Trial:Prospective Randomized Evaluation of Celecoxib Integrated Safety vs. Ibuprofen Or Naproxen (PRECISION; NCT00346216)DesignThe PRECISION trial was a do

In [7]:
from langchain.tools import Tool
search_pharma_kb_tool = Tool(
    name="pharma_kb_search",
    func=search_pharma_kb,
    description="Pharamaceutical knowledge base"
)

In [8]:
qa_agent = create_react_agent(
    model=model,
    tools=[search_pharma_kb_tool],
    name="question-answer_expert",
    prompt="You are a pharmaceutical expert specializing in answering user questions about medication, composition, dosage, and side effects. Use the pharma knowledge base to retrieve relevant information and answer the user's questions in a detailed, natural language format. Provide references from the dataset when necessary. Just use the chunks from the knowledge to answer."
)

In [9]:
recommender_agent = create_react_agent(
    model=model,
    tools=[search_pharma_kb_tool],
    name="expert-recommender",
    prompt="You are a pharmaceutical expert in recommending medications. Based on user symptoms or conditions, recommend the appropriate medication while warning about possible harmful combinations or contraindications. Use the pharma knowledge base to retrieve relevant details and ensure your recommendations are safe and personalized. Just use the chunks from the knowledge to answer."
)

In [10]:
alternatives_agent = create_react_agent(
    model=model,
    tools=[search_pharma_kb_tool],
    name="alternatives-generator",
    prompt="You are an expert in pharmaceutical alternatives. Based on user queries, suggest safe alternatives for medications. Use the pharma knowledge base to ensure the alternatives are appropriate and explain any differences in effects, risks, and usage instructions. Just use the chunks from the knowledge to answer."
)

In [11]:
summarizer_agent = create_react_agent(
    model=model,
    tools=[search_pharma_kb_tool],
    name="product-summarizer",
    prompt="You are an expert at summarizing pharmaceutical products. Provide a concise and clear summary of the product's key details, including its purpose, administration instructions, side effects, and warnings. Use the pharma knowledge base for accurate information. Just use the chunks from the knowledge to answer."
)

In [12]:
workflow = create_supervisor(
    [qa_agent, recommender_agent, alternatives_agent, summarizer_agent],
    model=model,
    prompt=(
        """You are a supervisor managing four agents: a question-answer expert, a recommender, an alternatives generator, and a product summarizer. Based on the user's query, invoke the appropriate agent to retrieve the required information and provide detailed, well-structured answers or recommendations. 
        Ensure that:
        1. The response is concise and clearly structured.
        2. Unnecessary punctuation, such as excessive colons, dashes, or other non-essential symbols, is avoided.
        3. The information is easy to read and free of clutter.
        4. Provide the final answer in a well-formatted manner using bullet points, sections, or headings where necessary.
        5. Let the final answer be of text type, not a markdown or html type."""
    )
)

In [13]:
app = workflow.compile()

result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "What are some alternatives to Ibuprofen for pain relief"
        }
    ]
})

In [14]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

What are some alternatives to Ibuprofen for pain relief
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_alternatives-generator (call_5RmS5ZIEr55BgnL2p2KysfnF)
 Call ID: call_5RmS5ZIEr55BgnL2p2KysfnF
  Args:
================================= Tool Message =================================
Name: transfer_to_alternatives-generator

Successfully transferred to alternatives-generator
================================== Ai Message ==================================
Name: alternatives-generator

Here are some safe alternatives to Ibuprofen for pain relief:

1. **Celecoxib**:
   - **Effectiveness**: Celecoxib is effective for moderate to severe pain, providing relief within 60 minutes.
   - **Usage**: It is often used for conditions like dysmenorrhea and post-surgical pain.
   - **Risks**: As a COX-2 inhibitor, it may have a lower risk of g

In [20]:
from textwrap import dedent

pretty_msg = dedent(result["messages"][-1].content)
print(pretty_msg)

Here are some alternatives to Ibuprofen for pain relief:

1. **Celecoxib**:
   - Effective for moderate to severe pain.
   - Provides relief within 60 minutes.
   - Often used for conditions like dysmenorrhea and post-surgical pain.
   - Lower risk of gastrointestinal side effects compared to traditional NSAIDs, but may pose cardiovascular risks.

2. **Mefenamic Acid**:
   - Effective for treating primary dysmenorrhea.
   - Typically used for menstrual pain.
   - Requires assessment of pain severity and may need additional analgesics.
   - Similar gastrointestinal risks as other NSAIDs; use with caution in patients with a history of such issues.

### Key Differences:
- **Onset of Action**: Celecoxib may provide faster relief than mefenamic acid.
- **Side Effects**: Both have common NSAID side effects, but celecoxib may be easier on the stomach.
- **Indications**: Mefenamic acid is more focused on menstrual pain, while celecoxib has broader applications.

### Recommendations:
- Consult 